# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [45]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [46]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [48]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


15
[['', 'Logged In', 'Walter', 'M', '0', 'Frye', '', 'free', 'San Francisco-Oakland-Hayward, CA', 'GET', 'Home', '1.54092E+12', '38', '', '200', '1.54111E+12', '39'], ['', 'Logged In', 'Kaylee', 'F', '0', 'Summers', '', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'GET', 'Home', '1.54034E+12', '139', '', '200', '1.54111E+12', '8'], ["Des'ree", 'Logged In', 'Kaylee', 'F', '1', 'Summers', '246.30812', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'PUT', 'NextSong', '1.54034E+12', '139', 'You Gotta Be', '200', '1.54111E+12', '8'], ['', 'Logged In', 'Kaylee', 'F', '2', 'Summers', '', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'GET', 'Upgrade', '1.54034E+12', '139', '', '200', '1.54111E+12', '8'], ['Mr Oizo', 'Logged In', 'Kaylee', 'F', '3', 'Summers', '144.03873', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'PUT', 'NextSong', '1.54034E+12', '139', 'Flat 55', '200', '1.54111E+12', '8'], ['Tamba Trio', 'Logged In', 'Kaylee', 'F', '4', 'Summers', '177.18812', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'PUT', 'NextSon

In [49]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

12


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [50]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [51]:
#Create a Keyspace
query = """
    CREATE KEYSPACE IF NOT EXISTS sparkify_keyspace
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 } 
"""
session.execute(query)

#### Set Keyspace

In [52]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify_keyspace')

# Queries to ask following three questions of the data

### Query 1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### First, create a table with primary keys sessionId and itemInSession, where results are filtered based on sessionId

In [53]:
#Create a table that responds to query 1

query = """
    CREATE TABLE IF NOT EXISTS session_songs
    (sessionId int,itemInSession int,artist text, song_title text,song_length float,
    PRIMARY KEY (sessionID,itemInSession)) 
"""
session.execute(query)

#### Then, insert appropriate columns from the raw event data file into the table

In [54]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_songs (sessionId, itemInSession, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        artist_name, user_name, gender, itemInSession, user_last_name, length, level, location, sessionId, song, userId = line
        session.execute(query, (int(sessionId), int(itemInSession), artist_name, song, float(length)))

#### Do a SELECT to verify that the data have been inserted into each table

In [55]:
## SELECT statement to verify the data was entered into the table
rows = session.execute("""SELECT artist, song_title, song_length FROM session_songs WHERE sessionId = 338 AND itemInSession = 4""")

for row in rows:
    print(row.artist, row.song_title, row.song_length)

### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### First, create a table with primary keys userId, sessionId and itemInSession, where results are filtered by userId and sessionId and sorted by itemInSession

In [56]:
query = """
    CREATE TABLE IF NOT EXISTS songs_by_users
    (userId int, sessionId int, artist text, song text, firstName text, lastName text, itemInSession int,
    PRIMARY KEY((userId, sessionId), itemInSession))
    WITH CLUSTERING ORDER BY (itemInSession DESC)
    """
session.execute(query)

#### Then, insert appropriate columns from the raw event data file into the table

In [57]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_by_users (userId, sessionId, artist, song, firstName, lastName, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        session.execute(query, (int(userId), int(sessionId), artist, song, firstName, lastName, int(itemInSession)))

#### Do a SELECT to verify that the data have been inserted into each table

In [58]:
#SELECT statement to verify the data was entered into the table
rows = session.execute("""SELECT itemInSession, artist, song, firstName, lastName FROM songs_by_users WHERE userId = 10 AND sessionId = 182""")

for row in rows:
    print(row.iteminsession, row.artist, row.song, row.firstname, row.lastname)

### Query 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### First, create a table with primary keys song and userId

In [59]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "CREATE TABLE IF NOT EXISTS users_by_songs"
query = query + "(song text, firstName text, lastName text, userId int, PRIMARY KEY (song, userId))"
session.execute(query)

#### Then, insert appropriate columns from the raw event data file into the table

In [60]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_by_songs (song, firstName, lastName, userId)"
        query = query + " VALUES (%s, %s, %s, %s)"
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        session.execute(query, (song, firstName, lastName, int(userId)))

#### Do a SELECT to verify that the data have been inserted into each table

In [61]:
#SELECT statement to verify the data was entered into the table
rows = session.execute("""SELECT firstName, lastName FROM users_by_songs WHERE song = 'All Hands Against His Own'""")

for row in rows:
    print(row.firstname, row.lastname)

### Drop the tables before closing out the sessions

In [62]:
## Drop the table before closing out the sessions

session.execute("drop table session_songs")
session.execute("drop table songs_by_users")
session.execute("drop table users_by_songs")

### Close the session and cluster connection¶

In [63]:
session.shutdown()
cluster.shutdown()